In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pathlib
import sys
import json

In [3]:
# Get root directory
root_dir = pathlib.Path(os.getcwd()).parent.parent.parent.resolve()

# Add to Python path
sys.path.insert(0, str(root_dir))

# import packages from review_radar
from review_radar.repositories.review_repository import ReviewRepository
from review_radar.repositories.batch_repository import BatchRepository
from review_radar.services.labeling.providers.base_provider import LabelResult
from review_radar.services.labeling.providers.gemini_flash_lite_2_5_provider import GeminiFlashLite25Provider
from review_radar.utils import setup_logger

/Users/pitsanunamnil/Desktop/ai_sentiment/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/pitsanunamnil/Desktop/ai_sentiment/review_radar/services/labeling/providers/gemini_flash_lite_2_5_provider.py:5: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


In [4]:
logger = setup_logger("demo_use")
batch_repo = BatchRepository(client_type='supabase', logger=logger)
review_repo = ReviewRepository(client_type='supabase', logger=logger)
batch = 1

In [5]:
aspects = batch_repo.get_batch_aspects(batch_id=batch)
print(f"length of aspects: {len(aspects)}, aspects: {aspects}")


2025-12-31 10:40:47,057 - demo_use - INFO - BatchRepository: Fetching aspects for batch 1
2025-12-31 10:40:47,058 - demo_use - INFO - BatchDataSupabaseClient: Fetching aspects for batch 1
2025-12-31 10:40:47,577 - demo_use - INFO - Fetched 5 aspects for batch 1
length of aspects: 5, aspects: ['TASTE', 'PRICE', 'SERVICE', 'ATMOSPHERE', 'ACCESSIBILITY']


In [6]:
provider = GeminiFlashLite25Provider(aspects=aspects, logger=setup_logger("GeminiFlashLiteProvider"))

In [7]:
provider.get_model_info()

Model(name='models/gemini-2.5-flash-lite',
      base_model_id='',
      version='001',
      display_name='Gemini 2.5 Flash-Lite',
      description='Stable version of Gemini 2.5 Flash-Lite, released in July of 2025',
      input_token_limit=1048576,
      output_token_limit=65536,
      supported_generation_methods=['generateContent',
                                    'countTokens',
                                    'createCachedContent',
                                    'batchGenerateContent'],
      temperature=1.0,
      max_temperature=2.0,
      top_p=0.95,
      top_k=64)

In [8]:
print(provider.system_instructions)

Role: High-Precision Thai Restaurant ABSA Annotator
Task: Extract Score (S) and Confidence (C) for 5 aspects.

Aspects: TASTE, PRICE, SERVICE, ATMOSPHERE, ACCESSIBILITY

Rules for Confidence (C):
- Score (S) range: -1.0 (Very Negative) to 1.0 (Very Positive). If not mentioned, use null.
- If S is provided: C = Confidence that the score is accurate.
- If S is null: C = Confidence that this aspect is NOT mentioned in the text at all.
- C range: 0.0 to 1.0 (1.0 = Absolutely certain), Important: Not null.

Conflict Handling: If a review has mixed feedback (e.g., "Good food but oily"), provide a balanced Score (S) and lower the Confidence (C) to reflect the ambiguity.

Output Format: Strict JSON [[S1,S2,S3,S4,S5], [C1,C2,C3,C4,C5]]


In [9]:
reviews = review_repo.get_unlabeled_reviews(batch_id=batch, limit=1)

2025-12-31 10:40:47,978 - demo_use - INFO - Fetching unlabeled reviews for batch 1
2025-12-31 10:40:47,979 - demo_use - INFO - Fetching unlabeled reviews for batch 1
2025-12-31 10:40:48,112 - demo_use - INFO - Found 1 unlabeled reviews
2025-12-31 10:40:48,112 - demo_use - INFO - Found 1 unlabeled reviews


In [10]:
label_result: LabelResult = provider.process_label(reviews[0]['review'])

In [11]:
print(reviews[0]['review'])

ในย่านตลิ่งชันจะมีก๋วยเตี๋ยวตำลึงหลายเจ้ามาก บางเจ้าก็เก่าแก่ตัวจริง บางเจ้าก็เพิ่งมาเปิดที่หลัง ร้านนี้เพิ่งมาเปิดได้ช่วงหลังๆได้ทำเลดีตรงข้าม รร. มหรรณพาราม  ราคาถูกแต่ก็ให้น้อยตามราคาเตรียมสั่งเผื่อไว้ได้ล่วงหน้าได้เลย รสชาดต้มยำไม่จัดจ้านมาก โดยรวมก็พอใช้ได้ครับ


In [12]:
print(json.dumps(label_result.to_dict(), indent=2, ensure_ascii=False))

{
  "labels": {
    "TASTE": {
      "score": null,
      "confidence": 0.8
    },
    "PRICE": {
      "score": 0.1,
      "confidence": 0.9
    },
    "SERVICE": {
      "score": null,
      "confidence": 1.0
    },
    "ATMOSPHERE": {
      "score": null,
      "confidence": 1.0
    },
    "ACCESSIBILITY": {
      "score": null,
      "confidence": 1.0
    }
  },
  "metadata": {
    "usage_metadata": {
      "prompt_token_count": 334,
      "candidates_token_count": 33,
      "total_token_count": 369,
      "cached_content_token_count": 0
    },
    "text_len": 266,
    "model_version": "gemini-2.5-flash-lite"
  }
}
